# Quantum Chemistry Tutorial 3 - Excited states

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
January 2024

## Introduction

L'une des applications marquantes de la chimie quantique, tant dans le milieu universitaire que dans l'industrie, est l'étude de l'interaction de la lumière avec la matière. L'absorption (resp. émission) d'un photon par une molécule peut promouvoir (resp. rétrograder) un électron d'un état électronique inférieur (resp. supérieur) à un état électronique d'énergie supérieure (resp. inférieure). La longueur d'onde des photons (c'est-à-dire l'énergie) requise pour que ces transitions se produisent est déterminée par la différence entre les deux états électroniques respectifs. Par conséquent, il est impératif de pouvoir calculer avec précision les énergies des états fondamental et excité afin d'étudier les interactions lumière/matière. Ces différences énergétiques jouent un rôle central dans de nombreuses technologies telles que les panneaux solaires, les diodes électroluminescentes (DEL ou en anglais, light-emitting diodes (LED)), les écrans et les colorants.

![Abs_emiss.png](Graphics/Abs_emiss.png)

Pour être plus concret, un colorant doit émettre de la lumière dans une région étroite du spectre visible pour être approprié à son objectif, c'est-à-dire qu'il doit présenter une longueur d'onde spécifique. Un autre exemple est celui des panneaux solaires, où le spectre d'absorption d'une molécule est ajusté via une fonctionnalisation chimique pour s'adapter au spectre d'émission solaire afin d'optimiser l'efficacité de la production d'énergie. Nous montrons ici un exemple de spectre pour la molécule BODIPY ($C_9H_7BF_2N_2$, Boron Dipyrromethene), une molécule largement utilisée pour les colorants fluorescents. La molécule BODIPY absorbe la lumière à une longueur d'onde inférieure (énergie plus élevée) et émet de la lumière à une longueur d'onde plus élevée (énergie plus faible). Pour calculer ce spectre, il faut calculer les énergies des états fondamental et excité et calculer leurs intensités. Le spectre d'absorption du BODIPY le plus simple est présenté ci-dessous. Différentes longueurs d'onde d'absorption et d'émission peuvent être ciblées en remplaçant les atomes d'hydrogène par différents groupes fonctionnels [J. Chimique. Phys. 155, 244102 (2021)](https://aip.scitation.org/doi/10.1063/5.0076787).

![BODIPY](Graphics/bodipy_absorption.png)

Comme il existe un très grand nombre de composés à considérer, la prévision des spectres d'absorption/émission UV-visible serait un atout précieux pour la communauté scientifique.

Pour parvenir à une compréhension complète de l'interaction de la lumière avec une molécule, la communauté de la chimie quantique a travaillé sur plusieurs algorithmes. En général, il faut calculer les structures électroniques moléculaires pertinentes pour prédire l'absorption/l'émission de la lumière UV. Ce notebook montre comment PySFC permet les calculs d'états excités en implémentant l'algorithme la TDDFT-TDA. Le cas d'utilisation ici est Li $_2$ par souci de simplicité. Ce calcul être étendu à des systèmes beaucoup plus grands tels que la molécule BODIPY ci-dessus.

Il convient de noter que même avec tous les états excités calculés, des effets non triviaux peuvent se produire (effet de solvatation, changement de géométrie, etc.) dans lesquels tous modifient la forme d'un spectre. Dans ce notebook, nous n'expliquons pas comment ces effets sont pris en compte, mais les calculs présentés ici constituent les premières étapes nécessaires vers le calcul des états excités.


## Théorie de la fonctionnelle de la densité dépendante du temps - TDDFT

### Théorie de la fonction réponse

L'équation de Schrödinger dépendant du temps est la solution appropriée
équation du mouvement:

\begin{align}
  & \mathtt{H}|0(t)\rangle = \mathrm{i}\frac{\partial |0(t)\rangle}{\partial t},
  & \mathtt{H} = \mathtt{H}_0 + \mathtt{V}(t).
\end{align}
La perturbation $\mathtt{V}(t)$ est un opérateur à un électron périodique dans le temps, cad, $\mathtt{V}(t) = \mathtt{V}(t+T)$.
Plutôt que de résoudre la fonction d'état dépendant du temps, nous pouvons cibler les corrections ordre par ordre des *valeurs moyennes* dépendant du temps :
\begin{equation}
\begin{split}
\langle 0(t) |\mathtt{A} | 0(t) \rangle &=\langle 0 |\mathtt{A} | 0 \rangle
+\sum_{(b, B)} \langle \langle A; B \rangle \rangle_{\omega_{b}}
 \epsilon_{B}(\omega_{b})\exp[-\mathrm{i}\omega_{b}t] \\
&+   \frac12 \sum_{(b, B);(c, C)} \langle \langle A; B, C \rangle \rangle_{\omega_{b}, \omega_{c}}\epsilon_{B}(\omega_{b})\epsilon_{C}(\omega_{c})\exp[-\mathrm {i}(\omega_{b}+\omega_{c})t]   + \ldots
\end{split}
\end{equation}
Les coefficients de Fourier discrets sont les **fonctions de réponse** que nous pouvons mapper aux propriétés observables du système. Par exemple, la fonction de réponse linéaire :
\begin{equation}
\langle \langle A; B \rangle \rangle_{\omega} = \sum_{i \neq j} 
\frac{\langle i |\mathtt{A} |j\rangle\langle j|B|j\rangle}{\omega - \omega_{ij}}
-\frac{\langle i|B|j\rangle\langle j|\mathtt{A} |j\rangle}{\omega + \omega_{ij}}
\end{equation}
dans la base propre exacte de l'Hamiltonien non perturbé et il code *toutes* les énergies d'excitation du système (les **pôles**) et les propriétés de transition (les **résidus**).
L'expression somme sur états (SOS, sum-over-states) nécessite toute la base propre de l'Hamiltonien. Cependant, les propriétés de réponse peuvent être extraites sans connaissance préalable d’une base appropriée, à condition d’avoir accès au Hessien électronique moléculaire.

### TD-DFT-TDA

Dans les théories des champs auto-cohérentes (SCF), les énergies d'excitation des systèmes moléculaires peuvent être obtenu comme valeurs propres, $\omega_n$, du problème aux valeurs propres de la réponse :
$$
     \begin{pmatrix}
         \mathbf{A} & \mathbf{B} \\
         \mathbf{B}^\ast & \mathbf{A}^\ast
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X}_n \\ \mathbf{Y}_n
     \end{pmatrix} = \omega_n
     \begin{pmatrix}
         \mathbf{1} & \mathbf{0} \\
         \mathbf{0} & -\mathbf{1}
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X}_n \\ \mathbf{Y}_n
     \end{pmatrix},
$$    
où 
* $\mathbf{A}$ et $\mathbf{B}$ sont les orbitales les Hessiennes qui apparaissent également dans l'analyse de stabilité pour les états de référence,
\begin{equation}
\begin{split}
A_{aibj} &= \delta_{ij}f_{ab} - \delta_{ab}f_{ij} + (ai|jb) - \gamma (ab|ji) + w_{\mathrm{xc}; aijb} ,\\
B_{aibj} &= (ai|bj) - \gamma (aj|ib) + w_{\mathrm{xc}; aibj} ,
\end{split}
\end{equation}
avec $ij$ et $ab$ les orbitales occupées et virtuelles respectivement,
* $\omega_n$ est l'énergie d'excitation, et
* $\mathbf{X}_n$ et $\mathbf{Y}_n$ représentent la réponse de la matrice de densité ou l'opérateur statistique.

Dans les cas où le système possède un état fondamental dégénéré ou présente des instabilités triplet, les algorithmes utilisés pour résoudre le les équations ci-dessus peuvent être instables. Ceci peut être résolu en appliquant l'approximation de Tamm-Dancoff (TDA), où l'on néglige simplement les matrices $\mathbf{B}_n$ et $\mathbf{Y}_n$. Il en résulte un problème Hermitien aux valeurs propres
$$
\mathbf{AX}_n = \omega_n\mathbf{X}_n.
$$

Il est à noter que lorsque TDA est appliquée, dans la TDDFT, la réponse linéaire du potentiel de corrélation d'échange (XC) conduit à une dérivée d'ordre 2 de la fonctionnelle XC, qui n'apparaît pas dans la DFT de l'état fondamental.


## Propriétés des transitions

Les énergies d’excitation ne rendent compte ne sont pas suffisantes comprendre la physique du problème. Nous avons besoin de moments de transition pour comprendre si une excitation est autorisée (lumineuse) ou non (sombre).

Les énergies d'excitation et les vecteurs propres peuvent être utilisés pour calculer les moments de transition, tels que les moments dipolaires de transition électrique et magnétique, et les intensités spectroscopiques, telles que les forces d'oscillateur et les forces de rotation.
Par exemple, le moment dipolaire électrique de transition dans la jauge de longueur est :

$$
  f = \frac23 \omega_n \sum_{u=x,y,z}\sum_{ia}|(\mathbf{X}_n+\mathbf{Y}_n)_{ia}\mu_{ai, u}|^2.
$$


## Calculs avec PySCF

Le système moléculaire que nous utilisons comme exemple dans ce notebook est le Li $_2$ proche de sa géométrie d'équilibre [voir les données expérimentales sur la géométrie en phase gazeuse](https://cccbdb.nist.gov/expgeom2x.asp). 
<!-- Le calcul complet des énergies Li $_2$ serait non trivial et très coûteux en calcul ; nous nous limitons donc à un espace actif de 2 électrons sur 2 orbitales qui impliquent 4 qubits lorsqu'ils sont mappés sur un hamiltonien de qubits en utilisant la cartographie de Jordan-Wigner. Cependant, ce petit problème a encore des effets non triviaux, rendus particulièrement évidents dans la section [2.7] (#27). Nous définissons deux objets molécules :

- `mol_li2` défini comme la configuration de l'état fondamental avec 2 électrons dans le HOMO.
- `mol_li2_t` défini comme la configuration triplet avec un électron alpha dans chacun des HOMO et LUMO. -->

### Calculs DFT avec la prise en compte de la dispersion et du solvent

In [1]:
from pyscf import gto, scf, dft, tddft

mol_xyz = 'Li 0 0 0; Li 0 0 3.0' # Ref: https://cccbdb.nist.gov/expgeom2x.asp
mol = gto.Mole()
mol.build(
    atom = mol_xyz,  
    basis = '6-31g(d,p)',
    symmetry = True,
)

mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.kernel()
mf.analyze()

/home/taamangtchu/mambaforge/envs/qiskit-env/lib/python3.10/site-packages/pyscf/dft/libxc.py:1102: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -15.0123443659292
Wave-function symmetry id = 0
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux E2gx E2gy E2uy E2ux
                               2    0    0    1    0    0    0    0    0    0
**** MO energy ****
MO #1 (A1g #1), energy= -2.00064068373249 occ= 2
MO #2 (A1u #1), energy= -2.00056524182952 occ= 2
MO #3 (A1g #2), energy= -0.129344073811441 occ= 2
MO #4 (A1u #2), energy= -0.0549025752066176 occ= 0
MO #5 (E1uy #1), energy= -0.0229060330406868 occ= 0
MO #6 (E1ux #1), energy= -0.0229060330406868 occ= 0
MO #7 (A1g #3), energy= -0.0130314467468126 occ= 0
MO #8 (E1gx #1), energy= 0.0221211001313736 occ= 0
MO #9 (E1gy #1), energy= 0.0221211001313736 occ= 0
MO #10 (A1u #3), energy= 0.0670432709722944 occ= 0
MO #11 (A1g #4), energy= 0.104835858582985 occ= 0
MO #12 (E1uy #2), energy= 0.115765883514775 occ= 0
MO #13 (E1ux #2), energy= 0.115765883514775 occ= 0
MO #14 (A1g #5), energy= 0.140875174614694 occ= 0
MO #15 (E1gx #2), energy= 0.146347857475866 

((array([1.99954536e+00, 9.83322257e-01, 1.69538208e-03, 7.44373994e-36,
         6.14736948e-36, 9.77563437e-03, 7.63069316e-35, 4.33767670e-34,
         5.24425576e-03, 8.03846546e-36, 1.12441380e-35, 4.17108251e-04,
         9.04427697e-36, 1.33759982e-41, 1.99954536e+00, 9.83322257e-01,
         1.69538208e-03, 3.70767660e-35, 1.16281682e-35, 9.77563437e-03,
         4.91176821e-36, 2.59620100e-36, 5.24425576e-03, 2.95899312e-35,
         8.66439882e-35, 4.17108251e-04, 1.00771290e-35, 5.50625852e-39]),
  array([0.00000000e+00, 1.33226763e-15])),
 array([0., 0., 0.]))

In [3]:
mf.dump_scf_summary()

**** SCF Summaries ****
Total Energy =                         -15.012344365929231
Nuclear Repulsion Energy =               1.587531632760000
One-electron Energy =                  -22.773640518640381
Two-electron Coulomb Energy =            9.874695457779069
DFT Exchange-Correlation Energy =       -3.700930937827920


### Calculs des états excités singulets avec la TDDFT

Effectuons les calculs sans et avec l'approximation TDA.

In [4]:
mytd = tddft.TDDFT(mf)
mytd.kernel(nstates = 10)
mytd.analyze(verbose=4)

Excited State energies (eV)
[1.97453504 2.72071312 2.72071313 2.81442662 3.21780081 3.21780081
 4.69411658 5.58014795 5.75497985 5.75497985]

** Singlet excitation energies and oscillator strengths **
Excited State   1:  A1u      1.97454 eV    627.92 nm  f=0.5099
       3 -> 4        -0.71952
Excited State   2:  ???      2.72071 eV    455.70 nm  f=0.5222
       3 -> 5        -0.51331
       3 -> 6        -0.49544
Excited State   3:  ???      2.72071 eV    455.70 nm  f=0.5222
       3 -> 5         0.49544
       3 -> 6        -0.51331
Excited State   4:  A1g      2.81443 eV    440.53 nm  f=0.0000
       3 -> 7         0.70577
Excited State   5:  ???      3.21780 eV    385.31 nm  f=0.0000
       3 -> 8         0.17116
       3 -> 9        -0.68606
Excited State   6:  ???      3.21780 eV    385.31 nm  f=0.0000
       3 -> 8        -0.65925
       3 -> 9         0.25567
Excited State   7:  A1u      4.69412 eV    264.13 nm  f=0.0071
       3 -> 4        -0.10133
       3 -> 10       -0.7025

In [5]:
mytda = tddft.TDA(mf)
mytda.kernel(nstates = 10)
mytda.analyze(verbose=4)

Excited State energies (eV)
[2.21611113 2.85924466 2.85924466 2.90621699 3.22079052 3.22079052
 4.78068875 5.6102661  5.78209158 5.78209158]

** Singlet excitation energies and oscillator strengths **
Excited State   1:  A1u      2.21611 eV    559.47 nm  f=0.8699
       3 -> 4         0.69358
       3 -> 10       -0.13053
Excited State   2:  ???      2.85924 eV    433.63 nm  f=0.7240
       3 -> 5        -0.60728
       3 -> 6         0.35651
Excited State   3:  ???      2.85924 eV    433.63 nm  f=0.7240
       3 -> 5         0.35651
       3 -> 6         0.60728
Excited State   4:  A1g      2.90622 eV    426.62 nm  f=0.0000
       3 -> 7        -0.69854
Excited State   5:  ???      3.22079 eV    384.95 nm  f=0.0000
       3 -> 8        -0.24124
       3 -> 9        -0.66455
Excited State   6:  ???      3.22079 eV    384.95 nm  f=0.0000
       3 -> 8         0.66455
       3 -> 9        -0.24124
Excited State   7:  A1u      4.78069 eV    259.34 nm  f=0.0798
       3 -> 4         0.1300

In [6]:
# Singlet excitation energies
ee_singlets = mytda.e

# Singlet oscillators strength
OS_singlets = mytda.oscillator_strength(gauge='length')

### Calculs des états excités triplets avec la TDDFT

Par défaut, seuls les états excités singulets sont calculés. Afin de calculer les états excités des triplets, il faut définir l'attribut *singlet* sur `False`.

In [7]:
mytda.singlet = False
mytda.kernel(nstates = 10)
mytda.analyze(verbose=4)

Excited State energies (eV)
[0.87598355 1.53621236 1.53621236 1.82055057 2.9364583  2.9364583
 4.35257372 5.25934195 5.55023071 5.55023071]

** Triplet excitation energies and oscillator strengths **
Excited State   1:  A1u      0.87598 eV   1415.37 nm  f=0.3652
       3 -> 4         0.70464
Excited State   2:  ???      1.53621 eV    807.08 nm  f=0.3805
       3 -> 5         0.66567
       3 -> 6         0.22744
Excited State   3:  ???      1.53621 eV    807.08 nm  f=0.3805
       3 -> 5         0.22744
       3 -> 6        -0.66567
Excited State   4:  A1g      1.82055 eV    681.03 nm  f=0.0000
       3 -> 7        -0.70243
Excited State   5:  ???      2.93646 eV    422.22 nm  f=0.0000
       3 -> 9        -0.70158
Excited State   6:  ???      2.93646 eV    422.22 nm  f=0.0000
       3 -> 8        -0.70158
Excited State   7:  A1u      4.35257 eV    284.85 nm  f=0.0035
       3 -> 10       -0.69826
Excited State   8:  A1g      5.25934 eV    235.74 nm  f=0.0000
       3 -> 11       -0.69

In [8]:
# Triplet excitation energies
ee_triplets = mytda.e

### Calculs des propriétés

In [9]:
import os
import numpy as np
from pyscf.data import nist

au2ev = nist.HARTREE2EV

# Index of HOMO and LUMO
lumo_idx = mf.mo_occ.tolist().index(0.)
homo_idx = lumo_idx - 1

# Calculate the gap Homo-LUMO
E_HOMO = mf.mo_energy[homo_idx]*au2ev
E_LUMO = mf.mo_energy[lumo_idx]*au2ev
E_g = abs(E_HOMO - E_LUMO)

# fluorescence energy
f_energy = min(ee_singlets)

# Singlet-Tiplet gap
gap_ST = min(ee_singlets) - min(ee_triplets)

# Oscillator strength
OStr = OS_singlets[0]

# Lifetime calculation in seconds
tau = (f_energy * au2ev)**2 * OStr
tau = 2.3046E-8 / tau 

# Multi-Objective function
mobj = OStr - gap_ST * au2ev - np.abs(f_energy * au2ev - 3.2)


* DataFrame and saving in file


In [10]:
import pandas as pd

# Create the results dataframe
list_results_TDA = [E_HOMO, E_LUMO, E_g, f_energy* au2ev, gap_ST* au2ev, OStr, tau* 1E+9, mobj]
dict_results_TDA = {'eV/ns': list_results_TDA}

df_TDA = pd.DataFrame(dict_results_TDA,
                    index = ['Homo energy', 'LUMO energy', 'Gap energy', 
                             'Fluorescence energy',
                             'Singlet-Triplet gap',
                             'Oscillator strength',
                             'Lifetime',
                             'Multi-Obj'])

df_TDA

,eV/ns
Homo energy,-3.519632
LUMO energy,-1.493975
Gap energy,2.025656
Fluorescence energy,2.216111
Singlet-Triplet gap,1.340128
Oscillator strength,0.869859
Lifetime,5.394592
Multi-Obj,-1.454158


## Spectre UV/Vis

Dans cette section, nous allons calculer les énergies d'excitation de la molécule 1,2-oxazole ($C_3H_3NO$) et tracer son spectre UV/Vis. Il est à noter que la molécule 1,2-oxazole peut être utile comme pesticide, fongicide agrochimique et insecticide.

### Diagramme de Jablonski

Dans ce chapitre, nous aborderons les transitions entre l’état fondamental et les états excités de valence résultant de l’absorption des UV ou de la lumière visible. 
Les énergies photoniques de la lumière UV et visible sont à la même échelle que les différences d'énergie entre les états électroniques de valence, qui sont ainsi sondées, révélant des informations sur la structure électronique dans la région de valence et sur la structure fine vibrationnelle. Comme l'illustre le diagramme de Jablonski ci-dessous, les transitions entre les différents états se produisent de différentes manières et à des échelles de temps très différentes. Le diagramme illustre l'état fondamental (dans ce cas un état singulet, S$_0$), trois états excités singulet (S$_1$, S$_2$ et S$_3$), ainsi qu'un état excité triplet ( T$_1$). Chaque état électronique possède une structure vibrationnelle fine comme représenté schématiquement sur la figure.


![figure-jablonski](Graphics/jablonski_diagram.png)

Ce diagramme de Jablonski illustre les transitions radiatives et non radiatives possibles entre les états excités de valence singulet (S) et triplet (T). Ces transitions s'opèrent sur des échelles de temps très différentes : 
* $10^{-15}$ pour l'**absorption**, 
* $10^{-14}$ - $10^{-11}$ pour la **relaxation vibratoire et la conversion interne**, 
* $10^{-9} $ - $10^{-7}$ pour la **fluorescence**, 
* $10^{-8}$ - $10^{-3}$ pour le **croisement intersystème** et 
* $10^{-4}$ - $10^{-1} $ pour la **phosphorescence**.

L'**absorption** d'un photon UV ou visible entraîne généralement une excitation électronique de l'état fondamental à l'un des états excités singulet. L'absorption des photons est un processus ultrarapide qui se déroule à l'échelle de la femtoseconde (fs). Habituellement, le système est amené à un état d'excitation vibratoire de l'état excité électroniquement, car le moment dipolaire de transition pour un tel processus est plus grand qu'une transition vers l'état vibratoire fondamental (une transition 0-0).

Le retour complet à l’état fondamental implique différents processus qui fonctionnent à différentes échelles de temps. La **relaxation** d'un état vibratoirement excité à l'état vibratoire fondamental est un processus non radiatif qui se produit sur l'échelle de temps $10^{-14}$ - $10^{-11}$. Un autre processus non radiatif qui fonctionne sur la même échelle de temps est la **conversion interne**, où un état excité singulet passe à un autre via un point de croisement où deux surfaces d'énergie potentielle d'état excité se croisent. La transition non radiative d'un état excité singulet à un état excité triplet s'effectue également via un point d'intersection, un processus appelé **croisement intersystème** avec un taux beaucoup plus faible que la conversion interne ($10^{-8}$ - $10^{-3 }$s). Enfin, la transition radiative d'un état excité singulet vers le singulet est appelée **fluorescence** et opère sur l'échelle de temps $10^{-9}$ - $10^{-7}$, tandis que la transition entre un état excité triplet et le sol l'état donne lieu à la **phosphorescence** ($10^{-4}$ - $10^{-1}$ s).

In [11]:
import py3Dmol

# PySCF molecule object
mol_xyz = """  
  C      1.0701      0.4341     -0.0336
  C      0.8115     -0.9049     -0.1725
  C     -0.6249     -1.0279     -0.0726
  H      1.9842      1.0231     -0.0364
  H      1.5156     -1.7176     -0.3255
  H     -1.2289     -1.9326     -0.1322
  O     -0.0873      1.1351      0.1422
  N     -1.1414      0.1776      0.1122"""

mol = gto.M(atom = mol_xyz,
  basis = '6-31g(d,p)', 
  verbose=3)

# 3D representation
xyz_view = py3Dmol.view(width=300,height=300)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

* Définissons une distribution Gaussienne ($e^{-\frac{(x-\mu)^2}{2\sigma^2}}$) qui est un moyen approximatif de se rapprocher de la forme de la ligne pour chaque excitation. 

In [12]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


* Créons une fonction pour calculer l'état fondamental avec la DFT, et les états excités avec la TD-DFT et effectuer l'analyse spectrale. 

In [13]:
def run_spectral_analysis(mol, xc="lda"):
    n_states=10
    spectral_width=0.1

    # Ground State DFT
    mf = dft.RKS(mol, xc=xc).run()

    # Excited State DFT
    mytd = tddft.TDDFT(mf)
    mytd.nstates = n_states
    mytd.max_space = 100
    mytd.max_cycle = 200
    mytd.kernel();
    mytd.analyze()
    osc_strengths = mytd.oscillator_strength()[:n_states-5]

    # Convolve lineshapes to make spectra
    energies_ev = mytd.e[:n_states-5] * au2ev
    x_range = np.linspace(energies_ev.min()*0.9, energies_ev.max()*1.1, num=1000)
    intensity = np.zeros(x_range.size)

    for e, f in zip(energies_ev, osc_strengths):
        intensity += gaussian(x_range, e, spectral_width) * f

    # Rough Normalization
    dx = (x_range[-1] - x_range[0])/x_range.size
    area = (intensity*dx).sum()
    intensity /= area


    return x_range, intensity

In [14]:
import time

data = {"Excitation Energy (eV)":[], "Intensity":[], "Exchange-Correlation Functional":[]}

xcs = ["pbe", "B3LYP", "wB97X_V"]

for xc in xcs:
    ti = time.time()
    x_range, intensity = run_spectral_analysis(mol, xc=xc)

    data["Excitation Energy (eV)"] += x_range.tolist()
    data["Intensity"] += intensity.tolist()
    data["Exchange-Correlation Functional"] += [xc]*x_range.size
    tf = time.time()
    print(f"Time for {xc.upper()} calculations: {tf-ti:.2f}\n")

df = pd.DataFrame(data)

converged SCF energy = -245.753731392922
Excited State energies (eV)
[5.79194969 6.20360472 6.88154996 7.37275965 7.63820973 8.18584349
 8.26403538 8.59152772 8.65747716 8.68403139]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      5.79195 eV    214.06 nm  f=0.0046
Excited State   2:      6.20360 eV    199.86 nm  f=0.0616
Excited State   3:      6.88155 eV    180.17 nm  f=0.0298
Excited State   4:      7.37276 eV    168.17 nm  f=0.0033
Excited State   5:      7.63821 eV    162.32 nm  f=0.0007
Excited State   6:      8.18584 eV    151.46 nm  f=0.0592
Excited State   7:      8.26404 eV    150.03 nm  f=0.0008
Excited State   8:      8.59153 eV    144.31 nm  f=0.0001
Excited State   9:      8.65748 eV    143.21 nm  f=0.0728
Excited State  10:      8.68403 eV    142.77 nm  f=0.0000
Time for PBE calculations: 25.55

converged SCF energy = -246.02970847716
TD-SCF states [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] not converged.
Excited State energies (eV)
[4.47575969 6.08

In [15]:
import plotly.express as px
fig = px.line(df, x="Excitation Energy (eV)", y="Intensity", markers=True, color="Exchange-Correlation Functional")
fig.show()